In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import gc
import json
import math


import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.preprocessing import OneHotEncoder
from tqdm import tqdm
from sklearn.decomposition import PCA
from pandas_profiling import ProfileReport
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor, HistGradientBoostingClassifier
from lightgbm import LGBMClassifier

import os

%matplotlib inline

In [ ]:

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Loading Train and Test Data
train = pd.read_csv("../input/tabular-playground-series-mar-2021/train.csv")
test = pd.read_csv("../input/tabular-playground-series-mar-2021/test.csv")
sample_submission = pd.read_csv("../input/tabular-playground-series-mar-2021/sample_submission.csv") 

In [ ]:
y = train.target
train = train.drop(["id", "target"], axis=1)

test = test.drop(["id"], axis=1)

In [ ]:
xgb_test_preds_0 = np.load('/kaggle/input/tps-03-21-feature-importance-with-xgboost-and-shap/xgb_test_preds_0.npy')
xgb_train_oof_0 = np.load('/kaggle/input/tps-03-21-feature-importance-with-xgboost-and-shap/xgb_train_oof_0.npy')
xgb_test_preds_1 = np.load('/kaggle/input/tps-03-21-feature-importance-with-xgboost-and-shap/xgb_test_preds_1.npy')
xgb_train_oof_1 = np.load('/kaggle/input/tps-03-21-feature-importance-with-xgboost-and-shap/xgb_train_oof_1.npy')
test_preds_lr_0 = np.load('/kaggle/input/tps-mar-2021-eda/test_preds_lr_0.npy')
train_oof_lr_0 = np.load('/kaggle/input/tps-mar-2021-eda/train_oof_lr_0.npy')
test_preds_hgb_0 = np.load('/kaggle/input/tps-mar-2021-eda/test_preds_hgb_0.npy')
train_oof_hgb_0 = np.load('/kaggle/input/tps-mar-2021-eda/train_oof_hgb_0.npy')
test_preds_hgb_1 = np.load('/kaggle/input/tps-mar-2021-eda/test_preds_hgb_1.npy')
train_oof_hgb_1 = np.load('/kaggle/input/tps-mar-2021-eda/train_oof_hgb_1.npy')
test_preds_lgbm_0 = np.load('/kaggle/input/tps-mar-2021-eda/test_preds_lgbm_0.npy')
train_oof_lgbm_0 = np.load('/kaggle/input/tps-mar-2021-eda/train_oof_lgbm_0.npy')
test_preds_lgbm_1 = np.load('/kaggle/input/tps-mar-2021-eda/test_preds_lgbm_1.npy')
train_oof_lgbm_1 = np.load('/kaggle/input/tps-mar-2021-eda/train_oof_lgbm_1.npy')
test_preds_lgbm_2 = np.load('/kaggle/input/tps-mar-2021-eda/test_preds_lgbm_2.npy')
train_oof_lgbm_2 = np.load('/kaggle/input/tps-mar-2021-eda/train_oof_lgbm_2.npy')
h2o_automl_submission_6 = pd.read_csv('../input/mar-21-tps-h2o-automl/h2o_automl_submission_6.csv')

In [ ]:
roc_auc_score(y, xgb_train_oof_0)

In [ ]:
roc_auc_score(y, xgb_train_oof_1)

In [ ]:
roc_auc_score(y, train_oof_lgbm_2)

In [ ]:
roc_auc_score(y, 0.5*train_oof_lgbm_0+0.5*train_oof_lgbm_1)

In [ ]:
roc_auc_score(y, 0.7*train_oof_lgbm_0+0.7*train_oof_lgbm_1-0.4*train_oof_lgbm_2)

In [ ]:
roc_auc_score(y, 0.85*(0.7*train_oof_lgbm_0+0.7*train_oof_lgbm_1-0.4*train_oof_lgbm_2)+0.15*xgb_train_oof_0)

In [ ]:
0.8973398604513213

In [ ]:
roc_auc_score(y, -0.1*train_oof_hgb_0+1.1*xgb_train_oof_0)

In [ ]:
roc_auc_score(y, 1.05*(0.85*(0.7*train_oof_lgbm_0+0.7*train_oof_lgbm_1-0.4*train_oof_lgbm_2)+0.15*xgb_train_oof_0)-0.05*train_oof_hgb_0)

In [ ]:
roc_auc_score(y, train_oof_lr_0)

In [ ]:
roc_auc_score(y, 0.1*train_oof_lr_0+0.9*xgb_train_oof_0)

In [ ]:
0.8951630390961065

In [ ]:
roc_auc_score(y, 1.04*(1.08*(0.84*(0.7*train_oof_lgbm_0+0.7*train_oof_lgbm_1-0.4*train_oof_lgbm_2)+0.16*xgb_train_oof_0)-0.08*train_oof_hgb_0)-0.04*train_oof_lr_0)

In [ ]:
0.8973126931822996

In [ ]:
y_test = 1.04*(1.08*(0.84*(0.7*test_preds_lgbm_0+0.7*test_preds_lgbm_1-0.4*test_preds_lgbm_2)+0.16*xgb_test_preds_0)-0.08*test_preds_hgb_0)-0.04*test_preds_lr_0

In [ ]:
sample_submission['target'] = y_test
sample_submission.to_csv('submission_0.csv', index=False)
sample_submission.head()

In [ ]:
sample_submission['target'] = 1.05*y_test-0.05*h2o_automl_submission_6['target'].values
sample_submission.to_csv('submission_1.csv', index=False)
sample_submission.head()

In [ ]:
y_test = 0.95*(1.04*(1.08*(0.84*(0.52*test_preds_lgbm_0+0.48*test_preds_lgbm_1)+0.16*xgb_test_preds_0)-0.08*test_preds_hgb_0)-0.04*test_preds_lr_0)+0.05*xgb_test_preds_1
sample_submission['target'] = y_test
sample_submission.to_csv('submission_2.csv', index=False)
sample_submission.head()

In [ ]:
y_test = 1.05*(1.04*(1.08*(0.84*(0.52*test_preds_lgbm_0+0.48*test_preds_lgbm_1)+0.16*xgb_test_preds_0)-0.08*test_preds_hgb_0)-0.04*test_preds_lr_0)-0.05*xgb_test_preds_1
sample_submission['target'] = y_test
sample_submission.to_csv('submission_3.csv', index=False)
sample_submission.head()